In [2]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.4 MB/s eta 0:00:00


In [3]:
import os
import music21
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
import pickle
from collections import defaultdict, Counter
import json
import pandas as pd
from fractions import Fraction
from scipy.stats import mode
from itertools import combinations
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Dropout, Bidirectional, BatchNormalization, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import LambdaCallback
import matplotlib.pyplot as plt

<ipython-input-3-7c59f0f54744>:103: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
from google.colab import drive


# Mount Google Drive
drive.mount('/content/gdrive')

# Define the folder path
folder_path = '/content/gdrive/My Drive/RestartGC/'

# Define the file path for the input file
input_file_path = os.path.join(folder_path, 'df_new.csv')  # or 'df_new.csv' for CSV file

# Load df_new
df_new = pd.read_csv(input_file_path)  # or pd.read_csv(input_file_path) for CSV file

# Print the first few rows of df_new
print(df_new.head())

Mounted at /content/gdrive
      artist              genre    song_title  \
0  Metallica          hard rock  Blitzkrieg.2   
1  Metallica              metal  Blitzkrieg.2   
2  Metallica  old school thrash  Blitzkrieg.2   
3  Metallica               rock  Blitzkrieg.2   
4  Metallica       thrash metal  Blitzkrieg.2   

                                            features  \
0  [('Instrument', 'Electric Guitar', 0.0), ('Tem...   
1  [('Instrument', 'Electric Guitar', 0.0), ('Tem...   
2  [('Instrument', 'Electric Guitar', 0.0), ('Tem...   
3  [('Instrument', 'Electric Guitar', 0.0), ('Tem...   
4  [('Instrument', 'Electric Guitar', 0.0), ('Tem...   

                   Instrument  \
0  [('Electric Guitar', 0.0)]   
1  [('Electric Guitar', 0.0)]   
2  [('Electric Guitar', 0.0)]   
3  [('Electric Guitar', 0.0)]   
4  [('Electric Guitar', 0.0)]   

                                               Tempo  \
0  [('250.0', 0.0), ('250.0', 0.0), ('250.0', 0.0...   
1  [('250.0', 0.0), ('250.0', 

In [5]:
df_new

,artist,genre,song_title,features,Instrument,Tempo,TimeSignature,Chord,Note,unique_feature_count,...,TimeSignature_freq,KeySignature_freq,Chord_freq,Note_freq,feature_cooccurrence,key_modulation,harmonic_complexity,melodic_complexity,rhythmic_complexity,instrument_diversity
0,Metallica,hard rock,Blitzkrieg.2,"[('Instrument', 'Electric Guitar', 0.0), ('Tem...","[('Electric Guitar', 0.0)]","[('250.0', 0.0), ('250.0', 0.0), ('250.0', 0.0...","[('3/4', 0.0), ('3/4', 0.0), ('3/4', 0.0)]","[('7.11', 0.5, 127, 144.5), ('7', 0.5, 127, 16...","[('G3', 1.0, 122, 6.0), ('D4', 0.5, 122, 7.0),...",5,...,0.001308,0.0,0.012642,0.976460,"Counter({frozenset({'Note'}): 2507680, frozens...",False,29,2235,1,1
1,Metallica,metal,Blitzkrieg.2,"[('Instrument', 'Electric Guitar', 0.0), ('Tem...","[('Electric Guitar', 0.0)]","[('250.0', 0.0), ('250.0', 0.0), ('250.0', 0.0...","[('3/4', 0.0), ('3/4', 0.0), ('3/4', 0.0)]","[('7.11', 0.5, 127, 144.5), ('7', 0.5, 127, 16...","[('G3', 1.0, 122, 6.0), ('D4', 0.5, 122, 7.0),...",5,...,0.001308,0.0,0.012642,0.976460,"Counter({frozenset({'Note'}): 2507680, frozens...",False,29,2235,1,1
2,Metallica,old school thrash,Blitzkrieg.2,"[('Instrument', 'Electric Guitar', 0.0), ('Tem...","[('Electric Guitar', 0.0)]","[('250.0', 0.0), ('250.0', 0.0), ('250.0', 0.0...","[('3/4', 0.0), ('3/4', 0.0), ('3/4', 0.0)]","[('7.11', 0.5, 127, 144.5), ('7', 0.5, 127, 16...","[('G3', 1.0, 122, 6.0), ('D4', 0.5, 122, 7.0),...",5,...,0.001308,0.0,0.012642,0.976460,"Counter({frozenset({'Note'}): 2507680, frozens...",False,29,2235,1,1
3,Metallica,rock,Blitzkrieg.2,"[('Instrument', 'Electric Guitar', 0.0), ('Tem...","[('Electric Guitar', 0.0)]","[('250.0', 0.0), ('250.0', 0.0), ('250.0', 0.0...","[('3/4', 0.0), ('3/4', 0.0), ('3/4', 0.0)]","[('7.11', 0.5, 127, 144.5), ('7', 0.5, 127, 16...","[('G3', 1.0, 122, 6.0), ('D4', 0.5, 122, 7.0),...",5,...,0.001308,0.0,0.012642,0.976460,"Counter({frozenset({'Note'}): 2507680, frozens...",False,29,2235,1,1
4,Metallica,thrash metal,Blitzkrieg.2,"[('Instrument', 'Electric Guitar', 0.0), ('Tem...","[('Electric Guitar', 0.0)]","[('250.0', 0.0), ('250.0', 0.0), ('250.0', 0.0...","[('3/4', 0.0), ('3/4', 0.0), ('3/4', 0.0)]","[('7.11', 0.5, 127, 144.5), ('7', 0.5, 127, 16...","[('G3', 1.0, 122, 6.0), ('D4', 0.5, 122, 7.0),...",5,...,0.001308,0.0,0.012642,0.976460,"Counter({frozenset({'Note'}): 2507680, frozens...",False,29,2235,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3617,Ben E. King,rock-and-roll,Stand By Me,"[('Tempo', '125.0', 0.0), ('TimeSignature', '4...",NaN,"[('125.0', 0.0)]","[('4/4', 0.0)]",NaN,NaN,2,...,0.500000,0.0,0.000000,0.000000,"Counter({frozenset({'Tempo', 'TimeSignature'})...",False,0,0,1,0
3618,Ben E. King,soul,Stand By Me,"[('Tempo', '125.0', 0.0), ('TimeSignature', '4...",NaN,"[('125.0', 0.0)]","[('4/4', 0.0)]",NaN,NaN,2,...,0.500000,0.0,0.000000,0.000000,"Counter({frozenset({'Tempo', 'TimeSignature'})...",False,0,0,1,0
3619,Quarterflash,album rock,Harden My Heart,"[('Instrument', 'Sampler', 0.0), ('Tempo', '12...","[('Sampler', 0.0)]","[('120.0', 0.0), ('120.0', 0.0)]","[('4/4', 0.0), ('4/4', 0.0)]","[('6.9.11.2', 4.0, 79, 24.0), ('7.11.2', 4.0, ...","[('E4', Fraction(4, 3), 79, 38.0), ('F#4', 0.2...",5,...,0.005102,0.0,0.232143,0.755102,"Counter({frozenset({'Note'}): 43660, frozenset...",False,91,296,1,1
3620,Quarterflash,new wave pop,Harden My Heart,"[('Instrument', 'Sampler', 0.0), ('Tempo', '12...","[('Sampler', 0.0)]","[('120.0', 0.0), ('120.0', 0.0)]","[('4/4', 0.0), ('4/4', 0.0)]","[('6.9.11.2', 4.0, 79, 24.0), ('7.11.2', 4.0, ...","[('E4', Fraction(4, 3), 79, 38.0), ('F#4', 0.2...",5,...,0.005102,0.0,0.232143,0.755102,"Counter({frozenset({'Note'}): 43660, frozenset...",False,91,296,1,1


In [10]:
# USING AUTO FINETUNING WITH RANDOM SEARCH.


df_new.fillna({
    'Instrument': 'Unknown',
    'Tempo': 'Unknown',
    'TimeSignature': 'None',
    'Chord': 'None',
    'Note': 'None',
    'Instrument_freq': 0,
    'Tempo_freq': 0,
    'TimeSignature_freq': 0,
    'Chord_freq': 0,
    'Note_freq': 0
}, inplace=True)


le_instrument = LabelEncoder()
df_new['Instrument'] = le_instrument.fit_transform(df_new['Instrument'])

le_tempo = LabelEncoder()
df_new['Tempo'] = le_tempo.fit_transform(df_new['Tempo'])

le_timesignature = LabelEncoder()
df_new['TimeSignature'] = le_timesignature.fit_transform(df_new['TimeSignature'])

le_chord = LabelEncoder()
df_new['Chord'] = le_chord.fit_transform(df_new['Chord'])

le_note = LabelEncoder()
df_new['Note'] = le_note.fit_transform(df_new['Note'])

le_artist = LabelEncoder()
df_new['artist_encoded'] = le_artist.fit_transform(df_new['artist'])

le_genre = LabelEncoder()
df_new['genre_encoded'] = le_genre.fit_transform(df_new['genre'])



def extract_temporal_features(features_list):
    durations = []
    velocities = []
    offsets = []

    for feature in features_list:
        # Check the length of the tuple and unpack accordingly
        if len(feature) == 5:
            _, _, duration, velocity, offset = feature
        elif len(feature) == 4:
            _, _, duration, velocity = feature
            offset = 0  # Default value if offset is not present
        elif len(feature) == 3:
            _, _, duration = feature
            velocity, offset = 0, 0  # Default values if velocity and offset are not present
        else:
            continue  # Skip if the tuple structure is not recognized

        durations.append(duration)
        velocities.append(velocity)
        offsets.append(float(offset))  # Convert Fraction to float

    return np.mean(durations), np.mean(velocities), np.mean(offsets)

df_new['features'] = df_new['features'].apply(lambda x: tuple(x))
# Apply the function to the 'features' column
df_new['feature_duration'], df_new['feature_velocity'], df_new['feature_offset'] = zip(*df_new['features'].apply(extract_temporal_features))

# Normalize the new temporal features
df_new['feature_duration'] = df_new['feature_duration'] / df_new['feature_duration'].max()
df_new['feature_velocity'] = df_new['feature_velocity'] / df_new['feature_velocity'].max()
df_new['feature_offset'] = df_new['feature_offset'] / df_new['feature_offset'].max()


# Define the function to ensure all categorical columns are uniform
def uniform_column_type(df, column_names):
    for column in column_names:
        # Check if the column is of type 'object', which could suggest mixed types
        if df[column].dtype == object:
            # Apply a conversion of lists to string if the entry is a list
            df[column] = df[column].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
        # Additionally, if there are missing values (None), replace them with a string
        df[column] = df[column].fillna('None')

columns_to_encode = ['Instrument', 'Tempo', 'TimeSignature', 'Chord', 'Note', 'genre_encoded',
                     'feature_duration', 'feature_velocity', 'feature_offset']

uniform_column_type(df_new, columns_to_encode)


# Include these additional features in the input set
columns_to_encode += ['feature_duration', 'feature_velocity', 'feature_offset']

# Define the number of features and categories
NUM_FEATURES = len(columns_to_encode)
SEQUENCE_LENGTH = 32  # Adjust based on musical considerations
NUM_CATEGORIES = df_new['Note'].nunique() + df_new['Chord'].nunique() + df_new['Instrument'].nunique()

# Prepare the input and target sequences

input_sequences = []
target_sequences = []
for i in range(0, len(df_new) - SEQUENCE_LENGTH):
    sequence_input = df_new.iloc[i:i+SEQUENCE_LENGTH][columns_to_encode]
    sequence_output = df_new.iloc[i+1:i+1+SEQUENCE_LENGTH][['Instrument', 'Chord', 'Note']]  # adjust as per target features

    # Ensure columns are encoded before applying to_categorical
    # Added check for numeric data type
    sequence_input_encoded = [to_categorical(seq, num_classes=df_new[colname].nunique())
                              for colname, seq in zip(sequence_input.columns, sequence_input.T.values)
                              if pd.api.types.is_numeric_dtype(sequence_input[colname])]

    sequence_output_encoded = [to_categorical(seq, num_classes=df_new[colname].nunique())
                               for colname, seq in zip(sequence_output.columns, sequence_output.T.values)
                               if pd.api.types.is_numeric_dtype(sequence_output[colname])]

    input_sequences.append(np.hstack(sequence_input_encoded))
    target_sequences.append(np.hstack(sequence_output_encoded))


input_sequences = np.array(input_sequences)
target_sequences = np.array(target_sequences)

X_train, X_val, y_train, y_val = train_test_split(input_sequences, target_sequences, test_size=0.2, random_state=42)

# Function to build the model (with hyperparameters)
def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(LSTM(hp.Int('input_units', min_value=32, max_value=512, step=32),
                               return_sequences=True, input_shape=(SEQUENCE_LENGTH, NUM_FEATURES))))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Bidirectional(LSTM(hp.Int('lstm_units', min_value=32, max_value=512, step=32), return_sequences=True)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(TimeDistributed(Dense(NUM_CATEGORIES, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#  tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=25,  # Number of different hyperparameter combinations to try
    executions_per_trial=1,
    directory='/content/gdrive/My Drive/RestartGC',
    project_name='music_gen_tuning'
)

# Start  hyperparameter tuning process
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

best_model = tuner.get_best_models(num_models=1)[0]

history = best_model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_val, y_val))


# Display the model summary
best_model.summary()

best_model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[checkpoint_callback, reduce_lr])
best_model.save('/content/gdrive/My Drive/RestartGC/best_hyperparam_model.h5')

Trial 25 Complete [00h 00m 39s]
val_accuracy: 0.13709957897663116

Best val_accuracy So Far: 0.5280292630195618
Total elapsed time: 00h 15m 55s
Epoch 1/15
45/45 [==============================] - 10s 68ms/step - loss: 4.0551 - accuracy: 0.4445 - val_loss: 3.9511 - val_accuracy: 0.4725
Epoch 2/15
45/45 [==============================] - 1s 30ms/step - loss: 3.9439 - accuracy: 0.4332 - val_loss: 3.8576 - val_accuracy: 0.4647
Epoch 3/15
45/45 [==============================] - 1s 30ms/step - loss: 3.8612 - accuracy: 0.4283 - val_loss: 3.7853 - val_accuracy: 0.4451
Epoch 4/15
45/45 [==============================] - 1s 30ms/step - loss: 3.7931 - accuracy: 0.4206 - val_loss: 3.7275 - val_accuracy: 0.4229
Epoch 5/15
45/45 [==============================] - 1s 30ms/step - loss: 3.7401 - accuracy: 0.4147 - val_loss: 3.6830 - val_accuracy: 0.4140
Epoch 6/15
45/45 [==============================] - 1s 30ms/step - loss: 3.6949 - accuracy: 0.4140 - val_loss: 3.6445 - val_accuracy: 0.3905
Epoch 7/1

NameError: ignored

In [16]:
# USING AUTO FINETUNING WITH BOYESIAN PROBA:

import keras_tuner as kt



# ... [previous code remains unchanged]

# Function to build the model (with hyperparameters)
def build_model(hp):
    model = Sequential()
    model.add(Bidirectional(LSTM(hp.Int('input_units', min_value=32, max_value=512, step=32),
                               return_sequences=True, input_shape=(SEQUENCE_LENGTH, NUM_FEATURES))))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Bidirectional(LSTM(hp.Int('lstm_units', min_value=32, max_value=512, step=32), return_sequences=True)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(TimeDistributed(Dense(NUM_CATEGORIES, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#   Bayesian Optimization tuner
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=30,  # Number of different hyperparameter combinations to try
    directory='/content/gdrive/My Drive/RestartGC',
    project_name='music_gen_bayesian_opt'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

best_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[checkpoint_callback, reduce_lr])
best_model.save('/content/gdrive/My Drive/RestartGC/best_hyperparam_model_bayesian.h5')



Trial 30 Complete [00h 00m 56s]
val_accuracy: 0.46635618805885315

Best val_accuracy So Far: 0.5173224210739136
Total elapsed time: 00h 20m 49s


ValueError: ignored

In [ ]:
from tensorflow.keras.models import load_model

# Load the model (replace with your model's filepath)
model_filepath = '/content/gdrive/My Drive/RestartGC/best_hyperparam_model.h5'
model = load_model(model_filepath)


In [ ]:
# Prepare your prompt
# Replace 'Your Artist' or 'Your Genre' with your actual artist or genre
prompt_artist = le_artist.transform(['Your Artist'])[0] if 'Your Artist' in le_artist.classes_ else None
prompt_genre = le_genre.transform(['Your Genre'])[0] if 'Your Genre' in le_genre.classes_ else None

# Create initial input sequence (you might need to adjust this based on how your model was trained)
initial_sequence = np.zeros((1, SEQUENCE_LENGTH, NUM_FEATURES))
if prompt_artist is not None:
    initial_sequence[0, :, df_new.columns.get_loc('artist_encoded')] = prompt_artist
if prompt_genre is not None:
    initial_sequence[0, :, df_new.columns.get_loc('genre_encoded')] = prompt_genre


In [ ]:
# Generate music
generated_sequence = model.predict(initial_sequence)


In [ ]:
# Assuming you have the inverse transformations ready
# Example for notes (adjust for other features like chords, instruments, etc.)
decoded_notes = [le_note.inverse_transform([np.argmax(note)])[0] for note in generated_sequence[0]]


In [ ]:
import IPython.display as ipd

# Function to convert sequence back to MIDI (you need to define this)
midi_data = sequence_to_midi(decoded_notes)

# Save to a MIDI file
midi_file = 'generated_music.mid'
midi_data.write(midi_file)

# Convert MIDI to WAV for playback (using a library like pydub or similar)
wav_file = 'generated_music.wav'
midi_to_wav(midi_file, wav_file)  # You need to define this function

# Play the audio
ipd.Audio(wav_file)
